In [107]:
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

%matplotlib inline

from gurobipy import *

In [108]:
path = "instances\InstanceBordeauxV1.xlsx"
# Employees
df_Workers = pd.read_excel(path, sheet_name=0, index_col='EmployeeName')
# Employees unvabilites
df_Workers_un = pd.read_excel(path, sheet_name=1)
# Task
df_Task = pd.read_excel(path, sheet_name=2, index_col='TaskId')
# Task unvabilites
df_Task_un = pd.read_excel(path, sheet_name=3, index_col='TaskId')

In [109]:
df_Workers

,Latitude,Longitude,Skill,Level,WorkingStartTime,WorkingEndTime
EmployeeName,,,,,,
Valentin,45.151218,-0.822093,Oenology,2,7:00am,5:00pm
Ambre,45.199575,-0.822093,Oenology,1,8:00am,6:00pm


In [110]:
df_Task

,Latitude,Longitude,TaskDuration,Skill,Level,OpeningTime,ClosingTime
TaskId,,,,,,,
T1,44.556549,-0.319392,60,Oenology,1,8:00am,6:00pm
T2,44.967501,-0.608685,60,Oenology,1,8:00am,6:00pm
T3,45.144215,-0.734257,60,Oenology,2,8:00am,6:00pm
T4,45.264808,-0.771789,60,Oenology,1,12:00am,6:00pm
T5,45.044423,-0.668761,60,Oenology,2,8:00am,6:00pm
T6,45.199575,-0.746208,60,Oenology,2,8:00am,6:00pm
T7,45.397698,-0.966819,60,Oenology,1,8:00am,6:00pm
T8,45.023479,-0.807213,60,Oenology,1,8:00am,6:00pm
T9,45.292914,-0.936536,75,Oenology,2,8:00am,12:00pm


In [111]:
df_Task_un

,Start,End
TaskId,,


In [112]:
# Employees
dict_Workers = df_Workers.to_dict('index')
# Employees unvabilites
dict_Workers_un = df_Workers_un.to_dict('index')
# Task
dict_Task = df_Task.to_dict('index')
# Task unvabilites
dict_Task_un = df_Task_un.to_dict('index')

In [113]:
dict_Workers

{'Valentin': {'Latitude': 45.15121765523164,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 2,
  'WorkingStartTime': '7:00am',
  'WorkingEndTime': '5:00pm'},
 'Ambre': {'Latitude': 45.19957452440505,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 1,
  'WorkingStartTime': '8:00am',
  'WorkingEndTime': '6:00pm'}}

In [114]:
dict_Task

{'T1': {'Latitude': 44.55654938342008,
  'Longitude': -0.3193922422375719,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T2': {'Latitude': 44.96750095217799,
  'Longitude': -0.6086852638150881,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T3': {'Latitude': 45.14421541464031,
  'Longitude': -0.7342570469020379,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T4': {'Latitude': 45.264808304867096,
  'Longitude': -0.7717887212411139,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '12:00am',
  'ClosingTime': '6:00pm'},
 'T5': {'Latitude': 45.044422793402624,
  'Longitude': -0.6687606009488057,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T6': {'Latitude': 45.19957452440505,
  'Longitude'

#### Sets

In [115]:
Workers = list(df_Workers.index)
Skills = list(df_Workers["Skill"].unique())
Tasks = list(df_Task.index)
Houses = {w: "HouseOf" + w for w in Workers}
# Pause set is define down

In [116]:
print('Workers: ', Workers)
print('Skills: ', Skills)
print('Tasks: ', Tasks)
print('Houses: ', Houses)

Workers:  ['Valentin', 'Ambre']
Skills:  ['Oenology']
Tasks:  ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10']
Houses:  {'Valentin': 'HouseOfValentin', 'Ambre': 'HouseOfAmbre'}


#### Parameters

level_skill: level of worker (in Workers) in some skill (of Skills)

In [117]:
VELOCITY = 50

In [118]:
def time_to_minutes(time_str):
    str_hour = time_str[:-2]
    am_pm = time_str[-2:]
    hour_str, minute = str_hour.split(':')
    hour = int(hour_str)
    if am_pm == 'pm':
        if hour == 12:
            return hour * 60 + int(minute)
        hour += 12
    return hour * 60 + int(minute)

def minutes_to_time(total_min):
    hour = int(total_min//60)
    min_ = int(total_min%60)
    am_pm = 'am'
    if hour > 12:
        am_pm = 'pm'
        hour -= 12
    if hour == 12:
        am_pm = 'pm'
    return f"{hour:02d}:{min_:02d}{am_pm}"

In [119]:
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[skill] = df_Workers.apply(lambda x: x['Level'] if x['Skill'] == skill else 0, axis=1)
print(df_aux)
l = df_aux.to_dict('index')
l

              Oenology
EmployeeName          
Valentin             2
Ambre                1


{'Valentin': {'Oenology': 2}, 'Ambre': {'Oenology': 1}}

In [120]:
# create de dictionary with the information of pauses
Pauses = {}
PauseNode = {}
a_pause = {}
b_pause = {}
d_pause = {}

for w in Workers:
    w_pauses_df = df_Workers_un[df_Workers_un['EmployeeName'] == w]
    if w_pauses_df.shape[0] == 0:
        Pauses[w] = []
    else:
        pause_list = []
        for i in range(w_pauses_df.shape[0]):
            pause_name = f'Pause{w}{i+1}' 
            pause_list.append(pause_name)
            PauseNode[pause_name] = (w_pauses_df.iloc[i,1], w_pauses_df.iloc[i,2])
            a_pause[pause_name] = time_to_minutes(w_pauses_df.iloc[i,3])
            b_pause[pause_name] = time_to_minutes(w_pauses_df.iloc[i,4])
            d_pause[pause_name] = b_pause[pause_name] - a_pause[pause_name]
        Pauses[w] = pause_list
print('Pauses: ', Pauses)

Pauses:  {'Valentin': [], 'Ambre': ['PauseAmbre1']}


In [121]:
# Opening time for taks i
a = df_Task.apply(lambda x: int(time_to_minutes(x['OpeningTime'])), axis=1).to_dict() | a_pause
print('a: ', a)
# Closing time for taks i
b = df_Task.apply(lambda x: int(time_to_minutes(x['ClosingTime'])), axis=1).to_dict() | b_pause
print('b: ', b)

# time worker w start working
alpha = df_Workers.apply(lambda x: int(time_to_minutes(x['WorkingStartTime'])), axis=1).to_dict()
print('alpha: ', alpha)
# time worker w end working
beta = df_Workers.apply(lambda x: int(time_to_minutes(x['WorkingEndTime'])), axis=1).to_dict()
print('beta: ', beta)

# Duration of the task i
d = df_Task['TaskDuration'].to_dict() | d_pause
print('d: ', d)
# Skill requierd by task i
s = df_Task['Skill'].to_dict()
print('s: ', s)

a:  {'T1': 480, 'T2': 480, 'T3': 480, 'T4': 720, 'T5': 480, 'T6': 480, 'T7': 480, 'T8': 480, 'T9': 480, 'T10': 480, 'PauseAmbre1': 720}
b:  {'T1': 1080, 'T2': 1080, 'T3': 1080, 'T4': 1080, 'T5': 1080, 'T6': 1080, 'T7': 1080, 'T8': 1080, 'T9': 720, 'T10': 1080, 'PauseAmbre1': 780}
alpha:  {'Valentin': 420, 'Ambre': 480}
beta:  {'Valentin': 1020, 'Ambre': 1080}
d:  {'T1': 60, 'T2': 60, 'T3': 60, 'T4': 60, 'T5': 60, 'T6': 60, 'T7': 60, 'T8': 60, 'T9': 75, 'T10': 60, 'PauseAmbre1': 60}
s:  {'T1': 'Oenology', 'T2': 'Oenology', 'T3': 'Oenology', 'T4': 'Oenology', 'T5': 'Oenology', 'T6': 'Oenology', 'T7': 'Oenology', 'T8': 'Oenology', 'T9': 'Oenology', 'T10': 'Oenology'}


In [122]:
# Level requierd by task i on the skill s
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[(skill)] = df_Task.apply(lambda x: x['Level'] if x['Skill'] == skill else 100, axis=1)
df_aux

,Oenology
TaskId,
T1,1
T2,1
T3,2
T4,1
T5,2
T6,2
T7,1
T8,1
T9,2


In [123]:
r = df_aux.to_dict('index')
r

{'T1': {'Oenology': 1},
 'T2': {'Oenology': 1},
 'T3': {'Oenology': 2},
 'T4': {'Oenology': 1},
 'T5': {'Oenology': 2},
 'T6': {'Oenology': 2},
 'T7': {'Oenology': 1},
 'T8': {'Oenology': 1},
 'T9': {'Oenology': 2},
 'T10': {'Oenology': 1}}

In [124]:
def read_lat_log(df, alias = None):
    nodes = {}
    for name, dic_inf in df.items():
        if alias:
            name = alias[name]
        nodes[name] = (dic_inf["Latitude"], dic_inf["Longitude"])
    return nodes

def haversine(pt1, pt2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [pt1[0], pt1[1], pt2[0], pt2[1]])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def distance_matrix(nodes):
    # Create distance matrix with haversine distance
    dist_matrix = {}
    for node_i in nodes.keys():
        dist_matrix[node_i] = {}
        for node_j in nodes.keys():
            dist = haversine(nodes[node_i], nodes[node_j])
            dist_matrix[node_i][node_j] = int(np.ceil((dist / VELOCITY) * 60))
    return dist_matrix


In [125]:
# Denfine the nodes
nodes = read_lat_log(dict_Workers, Houses) | read_lat_log(dict_Task) | PauseNode
nodes

{'HouseOfValentin': (45.15121765523164, -0.822092647754919),
 'HouseOfAmbre': (45.19957452440505, -0.822092647754919),
 'T1': (44.55654938342008, -0.3193922422375719),
 'T2': (44.96750095217799, -0.6086852638150881),
 'T3': (45.14421541464031, -0.7342570469020379),
 'T4': (45.264808304867096, -0.7717887212411139),
 'T5': (45.044422793402624, -0.6687606009488057),
 'T6': (45.19957452440505, -0.7462077931750715),
 'T7': (45.397697776585, -0.9668192708194538),
 'T8': (45.02347908679639, -0.8072126299796225),
 'T9': (45.29291368453335, -0.9365361007032236),
 'T10': (45.08146166752168, -0.8062453230620741),
 'PauseAmbre1': (44.7546825494, -0.6687606009488057)}

In [126]:
## Define the time matrix in minutes round ceil between a node (task or worker) with another (task or worker)
t = distance_matrix(nodes)
t['HouseOfValentin']['T1']

93

In [127]:
def init_param():
    print('Workers :', Workers) # These 4 are lists with the elements of the set.
    print('Skills :', Skills)
    print('Tasks :', Tasks)
    print('Houses :', Houses)   
    print('l :', l)             # Skill + level of workers
    print('a :', a)             # OpeningTime
    print('b :', b)             # ClosingTime
    print('alpha :', alpha)     # WorkingStartTime
    print('beta :', beta)       # WorkingEndTime
    print('d :', d)             # TaskDuration
    print('s :', s)             # Skill
    print('r :', r)             # Skill + level of tasks
    print('t :', t)             # distance_matrix

init_param()

Workers : ['Valentin', 'Ambre']
Skills : ['Oenology']
Tasks : ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10']
Houses : {'Valentin': 'HouseOfValentin', 'Ambre': 'HouseOfAmbre'}
l : {'Valentin': {'Oenology': 2}, 'Ambre': {'Oenology': 1}}
a : {'T1': 480, 'T2': 480, 'T3': 480, 'T4': 720, 'T5': 480, 'T6': 480, 'T7': 480, 'T8': 480, 'T9': 480, 'T10': 480, 'PauseAmbre1': 720}
b : {'T1': 1080, 'T2': 1080, 'T3': 1080, 'T4': 1080, 'T5': 1080, 'T6': 1080, 'T7': 1080, 'T8': 1080, 'T9': 720, 'T10': 1080, 'PauseAmbre1': 780}
alpha : {'Valentin': 420, 'Ambre': 480}
beta : {'Valentin': 1020, 'Ambre': 1080}
d : {'T1': 60, 'T2': 60, 'T3': 60, 'T4': 60, 'T5': 60, 'T6': 60, 'T7': 60, 'T8': 60, 'T9': 75, 'T10': 60, 'PauseAmbre1': 60}
s : {'T1': 'Oenology', 'T2': 'Oenology', 'T3': 'Oenology', 'T4': 'Oenology', 'T5': 'Oenology', 'T6': 'Oenology', 'T7': 'Oenology', 'T8': 'Oenology', 'T9': 'Oenology', 'T10': 'Oenology'}
r : {'T1': {'Oenology': 1}, 'T2': {'Oenology': 1}, 'T3': {'Oenology': 2}, 'T4

### Les variables

Some commentaries (from Francisco): 

1) it  necesary to put the option that workers do a trip from their house to their house, because we are making that them has to get out once and have to get bac once. If they do nothing in the optimal, they will pick this fictional arc that basicly means: do nothing.

2) 🤔 In the restriction "7- Border task sequence conditions" it says:

```python
ContrBorderSeqDeb = {(Houses[w], j, w): m.addConstr(alpha[w] + t[Houses[w]][j] <= T[j] + MT*(1 - X[(Houses[w], j, w)])) for j in Tasks for w in Workers}
ContrBorderSeqFin = {(i, Houses[w], w): m.addConstr(T[i] + d[i] + t[i][Houses[w]] <= beta[w] + MT*(1 - X[(i, Houses[w], w)])) for i in Tasks for w in Workers}
```

I think we could better write:

```python
ContrBorderSeqDeb = {(Houses[w], j, w): m.addConstr(alpha[w] + t[Houses[w]][j] <= T[j]) for j in Tasks for w in Workers}
ContrBorderSeqFin = {(i, Houses[w], w): m.addConstr(T[i] + d[i] + t[i][Houses[w]] <= beta[w]) for i in Tasks for w in Workers}
```


In [128]:
def init_param():
    return Workers, Skills, Tasks, l, a, b, alpha, beta, d, s, r, t

####################################
##    Initialisation du modèle    ##
####################################
m = Model("Phase 1")

####################################
##  Initialisation des variables  ##
####################################
# - X_{ijw} = 1 if worker w makes trip from i to j
#             0 otherwise
# - There are four cases: 1) i node and j node. 2) i node and j house. 3) i house and j node. 4) i house and j house.
#
# - Note that: it does not exist the trip from one node to the same node if node is not a house. And that workers
#              can not return to another worker's house
X = {(i, j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{j}") for w in Workers for j in Tasks + Pauses[w] for i in Tasks + Pauses[w] if j != i} |\
      {(i, Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{Houses[w]}") for w in Workers for i in Tasks + Pauses[w]} |\
          {(Houses[w], j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{j}") for w in Workers for j in Tasks + Pauses[w]} |\
              {(Houses[w], Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{Houses[w]}") for w in Workers}
               

# - T_i = time of begining of task i
T = {i: m.addVar(vtype=GRB.INTEGER, name=f"temps_début_tâche_{i}") for i in Tasks} | {p: a[p] for w in Workers for p in Pauses[w]}

## Variables additionnelles

Y = {(i, w): LinExpr(quicksum([X[(i, j, w)] for j in Tasks + Pauses[w] + [Houses[w]] if j != i])) for w in Workers for i in Tasks + Pauses[w]}
Y_bis = {(i, w): LinExpr(quicksum([X[(j, i, w)] for j in Tasks + Pauses[w] + [Houses[w]] if j != i])) for w in Workers for i in Tasks + Pauses[w]}

####################################
## Initialisation des contraintes ##
####################################

## 1- All tasks have to be done once ✅
ContrDone = {i: m.addConstr(quicksum([Y[(i, w)] for w in Workers]) == 1) for i in Tasks}

## 2 -Workers have to be capable of doing the Tasks ✅
MS = 10
ContrSkill = {(i, w, s):m.addConstr(r[i][s] <= l[w][s] + MS*(1 - Y[(i, w)])) for i in Tasks for w in Workers for s in Skills}

## 3- flow restriction ✅
ContrFlow = {(i, w): m.addConstr(Y[(i, w)] == Y_bis[(i, w)]) for w in Workers for i in Tasks + Pauses[w]}

## 4- Border flow conditions ✅
ContrBorderL = {w: m.addConstr(quicksum([X[(i, Houses[w], w)] for i in Tasks + Pauses[w] + [Houses[w]]]) == 1) for w in Workers}
ContrBorderR = {w: m.addConstr(quicksum([X[(Houses[w], j, w)] for j in Tasks + Pauses[w] + [Houses[w]]]) == 1) for w in Workers}

## 5- Task disponibility ✅
ContrTaskDisp = {i: m.addConstr(a[i] <= T[i]) for i in Tasks}
ContrTaskDisp = {i: m.addConstr(T[i] + d[i] <= b[i]) for i in Tasks}

## 6- task sequence is possible ✅
MT = 24*60
ContrSeq = {(i, j, w): m.addConstr(T[i] + d[i] + t[i][j] <= T[j] + MT*(1 - X[(i, j, w)])) for w in Workers for i in Tasks + Pauses[w] for j in Tasks + Pauses[w] if i != j}

## 7- Task sequence borders conditions ✅🤔
ContrBorderSeqDeb = {(Houses[w], j, w): m.addConstr(alpha[w] + t[Houses[w]][j] <= T[j] + MT*(1 - X[(Houses[w], j, w)])) for w in Workers for j in Tasks + Pauses[w]}
ContrBorderSeqFin = {(i, Houses[w], w): m.addConstr(T[i] + d[i] + t[i][Houses[w]] <= beta[w] + MT*(1 - X[(i, Houses[w], w)])) for w in Workers for i in Tasks + Pauses[w]}

## 8- Employees have unavailabilities
ContrPausDone = {i: m.addConstr(Y[(i, w)] == 1) for w in Workers for i in Pauses[w]}

####################################
##  Initialisation de l'objectif  ##
####################################
m.setObjective(quicksum([t[i][j]*X[(i, j, w)] for (i, j, w) in X.keys()]), GRB.MINIMIZE)

m.update()
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 318 rows, 254 columns and 1601 nonzeros
Model fingerprint: 0x9f0957e7
Variable types: 0 continuous, 254 integer (244 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 147 rows and 79 columns
Presolve time: 0.01s
Presolved: 171 rows, 175 columns, 836 nonzeros
Variable types: 0 continuous, 175 integer (165 binary)
Found heuristic solution: objective 322.0000000

Root relaxation: objective 2.163551e+02, 53 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  216.355

Comantaire (from Francisco):

I chequed the following solution and it is good in sequence concept and in respecting time (still not 100% chequed since i did not cheque considering the travel time, but in general i saw that it will be ok if there are in reality trajects that can be done in 10 minutes).

In [129]:
for (i, j, w) in X.keys():
    if X[(i, j, w)].x:
        if j in Tasks:
            print(X[(i, j, w)].VarName, "à", minutes_to_time(T[j].x))
        else:
             print(X[(i, j, w)].VarName)

Valentin_fait_le_trajet_T6_à_T3 à 02:18pm
Valentin_fait_le_trajet_T7_à_T4 à 12:00pm
Valentin_fait_le_trajet_T3_à_T5 à 03:33pm
Valentin_fait_le_trajet_T4_à_T6 à 01:10pm
Valentin_fait_le_trajet_T9_à_T7 à 09:30am
Ambre_fait_le_trajet_PauseAmbre1_à_T1 à 01:43pm
Ambre_fait_le_trajet_T1_à_T2 à 04:23pm
Ambre_fait_le_trajet_T10_à_T8 à 10:21am
Ambre_fait_le_trajet_T8_à_PauseAmbre1
Valentin_fait_le_trajet_T5_à_HouseOfValentin
Ambre_fait_le_trajet_T2_à_HouseOfAmbre
Valentin_fait_le_trajet_HouseOfValentin_à_T9 à 08:00am
Ambre_fait_le_trajet_HouseOfAmbre_à_T10 à 08:16am


In [133]:
print(['taskId', 'performed', 'employeeName', 'startTime'])

# all_rutes = {worker:['HouseOf'+str(worker)] for worker in Workers}
all_rutes = {worker:[] for worker in Workers}
for tasks in T.keys():
    result = [tasks]
    try:
        if T[tasks].x == 0:
            result.append(0)
            result += ['','']
        else:
            result.append(1)
            for (i, j, w) in X.keys():
                if i == tasks:
                    if X[(i, j, w)].x:
                        result.append(w)
                        all_rutes[w].append(result)
            result.append(int(T[tasks].x))
    except:
        if T[tasks] == 0:
            result.append(0)
            result += ['','']
        else:
            result.append(1)
            for (i, j, w) in X.keys():
                if i == tasks:
                    if X[(i, j, w)].x:
                        result.append(w)
                        all_rutes[w].append(result)
            result.append(T[tasks])
    print(result)

['taskId', 'performed', 'employeeName', 'startTime']
['T1', 1, 'Ambre', 823]
['T2', 1, 'Ambre', 983]
['T3', 1, 'Valentin', 858]
['T4', 1, 'Valentin', 720]
['T5', 1, 'Valentin', 933]
['T6', 1, 'Valentin', 790]
['T7', 1, 'Valentin', 570]
['T8', 1, 'Ambre', 621]
['T9', 1, 'Valentin', 480]
['T10', 1, 'Ambre', 496]
['PauseAmbre1', 1, 'Ambre', 720]


In [134]:
all_rutes

{'Valentin': [['T3', 1, 'Valentin', 858],
  ['T4', 1, 'Valentin', 720],
  ['T5', 1, 'Valentin', 933],
  ['T6', 1, 'Valentin', 790],
  ['T7', 1, 'Valentin', 570],
  ['T9', 1, 'Valentin', 480]],
 'Ambre': [['T1', 1, 'Ambre', 823],
  ['T2', 1, 'Ambre', 983],
  ['T8', 1, 'Ambre', 621],
  ['T10', 1, 'Ambre', 496],
  ['PauseAmbre1', 1, 'Ambre', 720]]}

In [135]:
from pprint import pprint
# sort the routes in order of time for each worker
for w in all_rutes.keys():
    all_rutes[w] = sorted(all_rutes[w], key=lambda x: x[3])

print('all_rutes: \n')
pprint(all_rutes)
# get the coordinates of the route for each worker
routes_lat_log = {worker:[] for worker in Workers}
for w in Workers:
    routes_lat_log[w].append(nodes['HouseOf'+w])
    routes_lat_log[w] += [nodes[task[0]] for task in all_rutes[w]]
    routes_lat_log[w].append(nodes['HouseOf'+w])
print('routes_lat_log: \n')
pprint(routes_lat_log)

all_rutes: 

{'Ambre': [['T10', 1, 'Ambre', 496],
           ['T8', 1, 'Ambre', 621],
           ['PauseAmbre1', 1, 'Ambre', 720],
           ['T1', 1, 'Ambre', 823],
           ['T2', 1, 'Ambre', 983]],
 'Valentin': [['T9', 1, 'Valentin', 480],
              ['T7', 1, 'Valentin', 570],
              ['T4', 1, 'Valentin', 720],
              ['T6', 1, 'Valentin', 790],
              ['T3', 1, 'Valentin', 858],
              ['T5', 1, 'Valentin', 933]]}
routes_lat_log: 

{'Ambre': [(45.19957452440505, -0.822092647754919),
           (45.08146166752168, -0.8062453230620741),
           (45.02347908679639, -0.8072126299796225),
           (44.7546825494, -0.6687606009488057),
           (44.55654938342008, -0.3193922422375719),
           (44.96750095217799, -0.6086852638150881),
           (45.19957452440505, -0.822092647754919)],
 'Valentin': [(45.15121765523164, -0.822092647754919),
              (45.29291368453335, -0.9365361007032236),
              (45.397697776585, -0.9668192708194

In [136]:
# create a map centered on the first point of the first route
m = folium.Map(location=nodes['HouseOf'+Workers[0]], zoom_start=10)

# add markers for each point in the first route
for node in nodes.values():
    folium.Marker(location=node).add_to(m)

# add a polyline to connect the markers for the second route
for route in routes_lat_log.values():
    color = '#{:06x}'.format(random.randint(0, 0xFFFFFF))
    folium.PolyLine(locations=route, color=color, weight=4).add_to(m)

# show the map
m


# Second version of the model without the variable $y$:

In [ ]:
####################################
##    Initialisation du modèle    ##
####################################
m = Model("Phase 1")

####################################
##  Initialisation des variables  ##
####################################
# - X_{ijw} = 1 if worker w makes trip from i to j
#             0 otherwise
# - There are four cases: 1) i node and j node. 2) i node and j house. 3) i house and j node. 4) i house and j house.
#
# - Note that: it does not exist the trip from one node to the same node if node is not a house. And that workers
#              can not return to another worker's house
X = {(i, j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{j}") for w in Workers for j in Tasks for i in Tasks if j != i} |\
      {(i, Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{Houses[w]}") for w in Workers for i in Tasks} |\
          {(Houses[w], j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{j}") for j in Tasks for w in Workers} |\
              {(Houses[w], Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{Houses[w]}") for w in Workers}

# - T_i = time of begining of task i
T = {i: m.addVar(vtype=GRB.INTEGER, name=f"temps_début_tâche_{i}") for i in Tasks}

####################################
## Initialisation des contraintes ##
####################################

## 1- All tasks have to be done once ✅
ContrDone = {i: m.addConstr(quicksum(X[(i, j, w)] for w in Workers for j in Tasks + [Houses[w]] if j != i) == 1) for i in Tasks}
## 2 -Workers have to be capable of doing the Tasks ✅
MS = 10
ContrSkill = {(i, w, s):m.addConstr(r[i][s] <= l[w][s] + MS*(1 - quicksum(X[(j, i, w)] for j in Tasks + [Houses[w]] if j != i))) for i in Tasks for w in Workers for s in Skills}

## 3- flow restriction ✅
ContrFlow = {(i, w): m.addConstr(quicksum(X[(i, j, w)] for j in Tasks + [Houses[w]] if j != i) == quicksum(X[(j, i, w)] for j in Tasks + [Houses[w]] if j != i)) for i in Tasks for w in Workers}

## 4- Border flow conditions ✅
ContrBorderL = {w: m.addConstr(quicksum(X[(i, Houses[w], w)] for i in Tasks + [Houses[w]]) == 1) for w in Workers}
ContrBorderR = {w: m.addConstr(quicksum(X[(Houses[w], j, w)] for j in Tasks + [Houses[w]]) == 1) for w in Workers}

## 5- Task disponibility ✅
ContrTaskDisp = {i: m.addConstr(a[i] <= T[i]) for i in Tasks}
ContrTaskDisp = {i: m.addConstr(T[i] + d[i] <= b[i]) for i in Tasks}

## 6- task sequence is possible ✅
MT = 24*60
ContrSeq = {(i, j, w): m.addConstr(T[i] + d[i] + t[i][j] <= T[j] + MT*(1 - X[(i, j, w)])) for i in Tasks for j in Tasks for w in Workers if i != j}

## 7- Border task sequence conditions ✅🤔
ContrBorderSeqDeb = {(Houses[w], j, w): m.addConstr(alpha[w] + t[Houses[w]][j] <= T[j]) for j in Tasks for w in Workers}
ContrBorderSeqFin = {(i, Houses[w], w): m.addConstr(T[i] + d[i] + t[i][Houses[w]] <= beta[w]) for i in Tasks for w in Workers}

## 8- Employees have unavailabilities
# TODO

####################################
##  Initialisation de l'objectif  ##
####################################
m.setObjective(quicksum(t[i][j]*X[(i, j, w)] for (i, j, w) in X.keys()), GRB.MINIMIZE)

m.update()
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 294 rows, 232 columns and 1444 nonzeros
Model fingerprint: 0x795e41aa
Variable types: 0 continuous, 232 integer (222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 136 rows and 72 columns
Presolve time: 0.01s
Presolved: 158 rows, 160 columns, 834 nonzeros
Variable types: 0 continuous, 160 integer (150 binary)

Root relaxation: objective 2.279472e+02, 43 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  227.94717    0   28          -  227.94717      -     - 

In [ ]:
for (i, j, w) in X.keys():
    if X[(i, j, w)].x:
        if j in Tasks:
            print(X[(i, j, w)].VarName, "à", minutes_to_time(T[j].x))
        else:
             print(X[(i, j, w)].VarName)

Valentin_fait_le_trajet_T6_à_T3 à 02:24pm
Valentin_fait_le_trajet_T7_à_T4 à 12:00pm
Valentin_fait_le_trajet_T3_à_T5 à 03:39pm
Valentin_fait_le_trajet_T4_à_T6 à 01:10pm
Valentin_fait_le_trajet_T9_à_T7 à 09:47am
Ambre_fait_le_trajet_T2_à_T1 à 10:39am
Ambre_fait_le_trajet_T1_à_T8 à 12:57pm
Ambre_fait_le_trajet_T8_à_T10 à 03:50pm
Valentin_fait_le_trajet_T5_à_HouseOfValentin
Ambre_fait_le_trajet_T10_à_HouseOfAmbre
Ambre_fait_le_trajet_HouseOfAmbre_à_T2 à 08:37am
Valentin_fait_le_trajet_HouseOfValentin_à_T9 à 08:17am


In [ ]:
print(['taskId', 'performed', 'employeeName', 'startTime'])

# all_rutes = {worker:['HouseOf'+str(worker)] for worker in Workers}
all_rutes = {worker:[] for worker in Workers}
for taks in T.keys():
    result = [taks]
    if T[taks].x == 0:
        result.append(0)
        result += ['','']
    else:
        result.append(1)
        for (i, j, w) in X.keys():
            if i == taks:
                if X[(i, j, w)].x:
                    result.append(w)
                    all_rutes[w].append(result)
        result.append(int(T[taks].x))
    print(result)

['taskId', 'performed', 'employeeName', 'startTime']
['T1', 1, 'Ambre', 639]
['T2', 1, 'Ambre', 517]
['T3', 1, 'Valentin', 864]
['T4', 1, 'Valentin', 720]
['T5', 1, 'Valentin', 939]
['T6', 1, 'Valentin', 790]
['T7', 1, 'Valentin', 587]
['T8', 1, 'Ambre', 777]
['T9', 1, 'Valentin', 497]
['T10', 1, 'Ambre', 950]


In [ ]:
all_rutes

{'Valentin': [['T3', 1, 'Valentin', 864],
  ['T4', 1, 'Valentin', 720],
  ['T5', 1, 'Valentin', 939],
  ['T6', 1, 'Valentin', 790],
  ['T7', 1, 'Valentin', 587],
  ['T9', 1, 'Valentin', 497]],
 'Ambre': [['T1', 1, 'Ambre', 639],
  ['T2', 1, 'Ambre', 517],
  ['T8', 1, 'Ambre', 777],
  ['T10', 1, 'Ambre', 950]]}

In [ ]:
from pprint import pprint
# sort the routes in order of time for each worker
for w in all_rutes.keys():
    all_rutes[w] = sorted(all_rutes[w], key=lambda x: x[3])

print('all_rutes: \n')
pprint(all_rutes)
# get the coordinates of the route for each worker
routes_lat_log = {worker:[] for worker in Workers}
for w in Workers:
    routes_lat_log[w].append(nodes['HouseOf'+w])
    routes_lat_log[w] += [nodes[task[0]] for task in all_rutes[w]]
    routes_lat_log[w].append(nodes['HouseOf'+w])
print('routes_lat_log: \n')
pprint(routes_lat_log)

all_rutes: 

{'Ambre': [['T2', 1, 'Ambre', 517],
           ['T1', 1, 'Ambre', 639],
           ['T8', 1, 'Ambre', 777],
           ['T10', 1, 'Ambre', 950]],
 'Valentin': [['T9', 1, 'Valentin', 497],
              ['T7', 1, 'Valentin', 587],
              ['T4', 1, 'Valentin', 720],
              ['T6', 1, 'Valentin', 790],
              ['T3', 1, 'Valentin', 864],
              ['T5', 1, 'Valentin', 939]]}
routes_lat_log: 

{'Ambre': [(45.19957452440505, -0.822092647754919),
           (44.96750095217799, -0.6086852638150881),
           (44.55654938342008, -0.3193922422375719),
           (45.02347908679639, -0.8072126299796225),
           (45.08146166752168, -0.8062453230620741),
           (45.19957452440505, -0.822092647754919)],
 'Valentin': [(45.15121765523164, -0.822092647754919),
              (45.29291368453335, -0.9365361007032236),
              (45.397697776585, -0.9668192708194538),
              (45.264808304867096, -0.7717887212411139),
              (45.1995745244050

In [ ]:
# create a map centered on the first point of the first route
m = folium.Map(location=nodes['HouseOf'+Workers[0]], zoom_start=10)

# add markers for each point in the first route
for node in nodes.values():
    folium.Marker(location=node).add_to(m)

# add a polyline to connect the markers for the second route
for route in routes_lat_log.values():
    color = '#{:06x}'.format(random.randint(0, 0xFFFFFF))
    folium.PolyLine(locations=route, color=color, weight=4).add_to(m)

# show the map
m